# Making 2d plot animation

### Import packages

In [1]:
import numpy                          as np
import matplotlib.pyplot              as plt
import matplotlib.animation           as ani
import IPython
%matplotlib agg

# import plons scripts
import plons
import plons.SmoothingKernelScript    as sk
import plons.PhysicalQuantities       as pq
import plons.ConversionFactors_cgs    as cgs
import plons.Plotting                 as plot

### Setting information about data

In [2]:
prefix = "wind"
loc = "/STER/matse/Papers/Esseldeurs+2023/Phantom/High/binary6Free/"
outputloc = "."
dump = loc+"wind_00600"

### Loading setup and dump

In [3]:
setup     = plons.LoadSetup(loc, prefix)

In [4]:
dumpData  = plons.LoadFullDump(dump, setup)

## Animation

I will create an animation with a 2D slice plot on the left, and a line slice on the right.

In [5]:
fig, ax = plt.subplots(1, figsize=(7, 7))
ax.set_aspect('equal')
ax.set_facecolor('k')

#### Creating 2D slice plot on the left

In [6]:
n = 500
x = np.linspace(-30*cgs.au, 30*cgs.au, n)
y = np.linspace(-30*cgs.au, 30*cgs.au, n)
X, Y = np.meshgrid(x, y)
Z    = np.zeros_like(X)
smooth_rot = sk.smoothMesh(X, Y, Z, dumpData, ['rho'])

mesh = ax.pcolormesh(X/cgs.au, Y/cgs.au, np.log10(smooth_rot["rho"]+1e-99), cmap=plt.cm.get_cmap('inferno'), vmin=-17, vmax = -14)
ax.set_xlim(x[0]/cgs.au, x[-1]/cgs.au)
ax.set_ylim(y[0]/cgs.au, y[-1]/cgs.au)

circleAGB, circleComp = plot.plotSink(ax, dumpData, setup)

#### Defining what to update in each frame

In each frame, an angle theta will be returned. Create a line at this angle, calculate the smoothed values, and plot them on the figures

In [7]:
def animate(frame):
    dump = loc+prefix+f"_%05i"%frame
    dumpData  = plons.LoadFullDump(dump, setup)
    smooth_rot = sk.smoothMesh(X, Y, Z, dumpData, ['rho'])

    mesh.set_array(np.log10(smooth_rot["rho"]+1e-99))
    circleAGB.center = dumpData['posAGB']/cgs.au
    circleComp.center = dumpData['posComp']/cgs.au

#### Creating the animation

By creating the animation, the frames will represent the angles from the previous function. Interval represents the delay between frames in milliseconds.

In [8]:
anim = ani.FuncAnimation(fig, animate, frames=range(1, 601), interval=100)
IPython.display.display(IPython.display.HTML(anim.to_html5_video()))
plt.close()